## Makefile

### 状态空间优化

#### 全局状态global_state

这个baseline只是粗暴地把所有states状态读入，但是当我仔细分析环境返回的信息时，我很快发现了问题：

如下是我手上目前拥有的东西,关于ASN和HPN，以及我的构想，会在下一条回复中出现,请你先了解如下内容:
----------------------------------------------------------------------

现在我手上拥有，大约400G的人类玩家游玩的真实数据(或者专家采样)，由主办方提供的baseline算法：BC克隆(读取真实数据，不与环境交互)和PPO(虽然与环境交互，但却离线策略的PPO，每次学习采样的buffer池，而且使用PPO截断)

但我注意到，这个baseline简单粗暴，具体做了这么一件事：
分别用一系列Embedding层(被包装在AgentStateLayer和GlobalStateLayer中)，将所有状态粗暴地转变为一个维度为share_layer_dim的特征向量，然后用这个特征向量输入PPO的actor和critic中，最终得到action和qvalue
```python
class Model(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.global_state_layer_dim = 64
        self.agent_state_layer_dim = 128
        self.global_state_layer = GlobalStateLayer()
        self.self_state_layer = AgentStateLayer()
        self.ally0_state_layer = AgentStateLayer()
        self.ally1_state_layer = AgentStateLayer()
        self.enemy0_state_layer = AgentStateLayer()
        self.enemy1_state_layer = AgentStateLayer()
        self.enemy2_state_layer = AgentStateLayer()
        # self.skill_layer = SkillLayer()
        self.share_layer_dim = self.global_state_layer_dim + self.agent_state_layer_dim * 6
        #-------------------🟠👆share_layer之前的所有部分，都是在处理和表示状态
        #而share_layer得到的，才是PPO中通用意义上给actor和critic的"state"
        self.share_layer = nn.Sequential(nn.Linear(self.share_layer_dim, 256), nn.ReLU(), nn.Linear(256, 128), nn.ReLU())
        self.value_layer = nn.Sequential(nn.Linear(128, 1))
        self.action_layer = nn.Sequential(nn.Linear(128, 52))
        self.opt = optim.Adam(self.parameters(), lr=1e-3)
```


一个典型的状态可以表示为以下形式：

```json
// 仅用作示例，非真实值
{
    "[member_id]": [
        // infos信息
        {
            // 射击奖励事件信息
            "shoot":{
                "self_id": 1,
                "event_id": 1,
                "position": 2,
                "me": 1,
                "ally": 0,
                "enemy": 0,
                "opponent": 0,
                "open_shoot": 1,
                ... // 其他信息
            },
            // 状态事件信息
            "state_event":{
                "free_ball": 1,
            },
            // 结束值
            "end_values":{
                "id": 1,
                "win": 0,
                "delta": 0,
                "score": 0,
                "team_two_try": 1,
                ... // 其他信息
            }
        },
        // 原始状态
        {
            "global_states":{
                "attack_remain_time": 10.0,
                "match_remain_time": 150.0,
                "is_home_team": 1,
                ... // 其他信息
            },
            "self_state":{
                "character_id": 1,
                "position_type": 0,
                "buff_key": 0,
                "buff_value": 0,
                "stature": 200,
                ... // 其他信息
            },
            "ally_0_state":{
                "character_id": 2,
                "position_type": 1,
                "buff_key": 0,
                "buff_value": 0,
                "stature": 200,
                ... // 其他信息
            },
            "ally_1_state":{
                "character_id": 3,
                "position_type": 2,
                "buff_key": 0,
                "buff_value": 0,
                "stature": 200,
                ... // 其他信息
            },
            "enemy_0_state":{
                "character_id": 4,
                "position_type": 3,
                "buff_key": 0,
                "buff_value": 0,
                "stature": 200,
                ... // 其他信息
            },
            "enemy_1_state":{
                "character_id": 5,
                "position_type": 4,
                "buff_key": 0,
                "buff_value": 0,
                "stature": 200,
                ... // 其他信息
            },
            "enemy_2_state":{
                "character_id": 6,
                "position_type": 5,
                "buff_key": 0,
                "buff_value": 0,
                "stature": 200,
                ... // 其他信息
            }
        },
        // 合法动作
        [1,1,1,0,0,...,0]
    ]
}
这是有关于这些问题的大致解读：
### 全局状态  属于`global_state`

|变量名称|描述|范围|
|-|-|-|
|attack_remain_time|剩余攻击时间|[0,20.0)|
|match_remain_time|剩余比赛时间|[0,150)|
|is_home_team|球员是否是主队||
|home_score|主队得分（当前分数）|0,2,3|
|away_score|客队得分||
|ball_position_x（y，z）|球的三维笛卡尔坐标。(拾取球，亦或者是拦阻敌人？)||
|vec_ball_basket_x（y，z）|篮球架和球之间的笛卡尔距离（投篮时机预判？）||
|team_own_ball|我们的球队是否有控球权（进攻）|1：是的   0：不|
|enemy_team_own_ball|敌方球队是否有控球权（防守）|1：是的   0：不|
|ball_clear|球是否越过三分线（二分球/三分球策略）|1：是的   0：不|
|ball_status|球的状态|0：没有   1：手动：发生在跳球时  （一开始双方争抢球） 2:球处在空闲状态（在地上）   3：射出：在投射球的过程中 4：拥有：拥有 球  5:传球：在传球的过程中|
|can_rebound|球是否可以抢篮板|1：是的   0：不|
|dis_to_rebound_x（z，y）|球员和篮板之间的笛卡尔距离||
|can_block|球是否可以被封堵（盖了帽了）|1：是的   0：不|
|shoot_block_pos_x（z）|射出球的盖帽的位置||
|dis_to_block_pos_x（z，y）|无法盖帽的位置信息||
|block_diff_angle（r）|盖帽的所偏差的角度和距离||

### 球员状态
### 代理状态 `self_state`

|变量名称|描述|范围|
|-|-|-|
|character_id|玩家编号||
|position_type|球员的位置类型|0：C  （中锋，顾名思义乃是一个球队的中心人物，又叫5号位 ） 1:PF （大前锋，篮球比赛中的一个重要球员位置，又叫4号位） 2：SF（SF:小前锋，乃是球队中最重要的得分者，又叫3号位）   3：SG  （得分后卫，在篮球场上是以得分为主要任务的司职，又叫2号位） 4:PG（控球又称组织后卫，是篮球比赛阵容中的一个固定位置，又叫1号位）|
|buff_key|Buff的字典键|增益的名称|
|buff_value|增益的字典值|增益的具体数值|
|身材|球员身材（厘米）||
|rational_shoot_distance|球员合理射门距离||
|position_x（y，z）|玩家的笛卡尔坐标||
|v_delta_x|玩家在 x 轴上的速度||
|v_delta_z|玩家在 z 轴上的速度。**篮球场平面由x轴和z轴组成**||
|player_to_me_dis_x（z）|其他玩家？相对于玩家的距离||
|basket_to_me_dis_x（z）|篮筐相对于玩家的坐标||
|ball_to_me_dis_x（z）|球相对玩家的坐标||
|polar_to_me_angle（r）|在极坐标玩家的角度和距离？||
|polar_to_basket_angle（r）|极坐标玩家的角度和距离？||
|facing_x（y，z）|玩家的面向矢量||
|block_remain_best_time|盖帽的最佳时间||
|block_remain_time|盖帽的持续时间||
|is_out_three_line|是否在三分线外||
|is_ball_owner|是否是球的持有者||
|own_ball_duration|拥有球的持续时间||
|cast_duration|投射时间||
|power|投射的力度||
|is_cannot_dribble|是否不能够再运球||
|is_pass_receiver|是否传给接收者||
|is_marking_opponent|是否接到了对手的球||
|is_team_own_ball|队伍是否拥有球||
|inside_defence|内测防守（在对手的干扰框内？）||
|player_state|玩家状态|0-5|
|skill_state|玩家施放的当前动作类型|0-26|

`player_state`:（对以上变量的说明）

- `0`：没有
- `1`：默认、站立或无球移动
- `2`：握住，握住球
- `3`：运球，运球
- `4`：施法技能，施法技能
- `5`：中断、刚性或受控
`skill_state`:

| 索引 |     描述     | 索引 |       描述      | 索引 |          描述          |
| :--: | :----------: | :--: | :------------: | :--: | :-------------------: |
|  0   |     无动作      |  9   |     背筐（一种扣篮战术）      |  18  |        背筐过人        |
|  1   | 通用动作（平常没有操作时的状态）    |  10  |     篮板      |  19  |        碰撞停车        |
|  2   |     盖帽     |  11  |   接球       |  20  |      潜水接球        |
|  3   |    卡位     |  12  |      盯防      |  21  |         粗鲁封盖         |
|  4   | 叫球      |  13  |      投篮      |  22  |          凿出（待定）          |
|  5   |  过人    |  14  |      抢断      |  23  |          疾跑扣篮          |
|  6   |    防守      |  15  |  即时动作    |  24  |         跳跃盖帽         |
|  7   |     传球     |  16  | 前抢断冷却    |  25  |    被迫停止动作    |
|  8   |    捡球      |  17  |  快速防守    |  26  | 切换机制动作 |

其中有关于动作action和事件Infos的内容，我会在下一条回复进行阐述

In [ ]:
这个baseline只是粗暴地把所有states状态读入，但是当我仔细分析环境返回的信息时，我很快发现了问题：

问题1： 有效的动作居然高达52个！
当我仔细研究后发现，其中12个动作是所有玩家的公共动作，其余的是每个玩家的技能信息：
"""
(这个表后跟的数字大概是移动的角度？)
| 索引 |   描述   | 索引 |       描述       |
| :--: | :------: | :--: | :-------------: |
|  0   |  无操作  |  6   |     移动: 45     |
|  1   | 移动: 90 |  7   |     移动: 225    |
|  2   |移动: 270 |  8   |     移动: 315    |
|  3   |移动: 180 |  9   |   取消技能   |
|  4   |  移动: 0  |  10  | 向盟友1传球 |
|  5   |移动: 135 |  11  | 向盟友2传球 |
"""

游戏一共有5位可以选择的球星，环境开始时，每边随机抽取3个，组成6个球员。
然而这些球员的技能和动作索引却完全不同！比如，如下是詹姆斯和库里的专属技能，技能索引值几乎完全不同：
"""
#### 詹姆斯-专属技能
| 索引 |       描述       | 索引 |        描述         | 索引 |           描述           |
| :--: | :--------------: | :--: | :-----------------: | :--: | :----------------------: |
|  12  |     向左突破     |  21  |   跑动空中接球1   |  30  |     低位之王(2分右侧)     |
|  13  |     向右突破     |  22  |     James的投篮     |  31  | 三分之王(三分左侧) |
|  14  |     呼唤球     |  23  | 坦克转身(左侧)  |  32  | 三分之王(三分右侧) |
|  15  |       掩护      |  24  | 坦克转身(右侧)  |  33  |    坦克扣篮(远距离)    |
|  16  |       防守       |  25  |     完全封盖     |  34  |   转身冲撞   |
|  17  |       篮板       |  26  |   集中三分球   |  35  |   单手扣篮   |
|  18  |       抢断       |  27  |     坦克冲撞     |  36  |   转身后仰投   |
|  19  |       盯防       |  28  |     突破跳投     |      |                         |
|  20  |      加速      |  29  |  低位之王(2分左侧)  |      |                         |

#### 库里-专属技能
| 索引 |    描述    | 索引 |         描述         | 索引 |         描述         |
| :--: | :--------: | :--: | :------------------: | :--: | :-----------------: |
|  12  |   向左突破   |  21  |   跑动空中接球1   |  30  |         冲刺         |
|  13  |   向右突破   |  22  |         加速         |  31  |       库有引力       |
|  14  |   呼唤球   |  23  |   跑动空中接球2   |  32  |         投篮         |
|  15  |     掩护     |  24  |         跳球         |  33  | 逆行运动(背后) |
|  16  |     防守     |  25  |      库里的投篮      |  34  | 逆行运动(左侧) |
|  17  |     篮板     |  26  |   坦克式转身(左)   |  35  | 逆行运动(右侧) |
|  18  |     抢断     |  27  |   坦克式转身(右)   |  36  |    转身&拉回    |
|  19  |     盖帽     |  28  |     突破后仰投     |  37  |      转身&投篮      |
|  20  |     盯防     |  29  | 低位之王(2分左侧) |  38  |   背后运球   |
"""

问题2：奖励函数过于单一，有大量infos尚未被利用！
在官方文档描述中，事件主要分为"state_event"(状态事件-即环境每次都返回的内容)，以及奖励事件(节点事件，只有触发了特定条件，才会被环境传回)
但是在baseline中，infos直接被忽视，rewards过于单一，缺乏infos和rewards的有效结合
"""
## 奖励事件

奖励事件主要分为两部分。一是与 各种关键节点相关的事件 ，例如投篮和抢断，每个事件的关键名称将在下表中呈现。另一部分是 与连续状态相关的事件 ，例如三分球没有出界。

### 节点事件

每个节点事件都有一些共同的信息，如下表所示：

| 特征键 |                  描述                   |
| :----: | :-------------------------------------: |
| self_id |                 玩家ID                  |
| event_id |                事件玩家ID               |
| position |           事件玩家的位置类型           |
|   me    |          事件玩家是否是自己            |
|  ally   |         事件玩家是否是盟友             |
|  enemy  |          事件玩家是否是敌人           |
| opponent | 事件玩家是否是我的对手玩家 |

请注意，您需要区分事件玩家和玩家本身。事件玩家指的是事件的主体，例如投篮事件。但是，游戏仍会将此投篮事件信息发送给每个玩家，以便用户在进行多代理相关处理时共享信息。

奖励事件主要包括以下几点：

* `score`: 计分事件，玩家射击并成功得分。
* `shoot`: 投篮事件，玩家尝试投篮但不一定进球。
* `steal`: 抢断事件，玩家尝试抢断对方。
* `block`: 盖帽事件，玩家尝试盖帽对方。
* `pick up`: 捡球事件，玩家尝试捡起球。
* `rebound`: 篮板事件，玩家尝试篮板。
* `screen`: 盯防事件，玩家尝试盯防并黏住对手。

每个事件的唯一特征如下表所示。请注意，“筛选”没有其自己的独特特征。

对以上几个事件具体内容的扩展
|  事件键  |   特征键   |                         描述                          |
| :------: | :-------: | :---------------------------------------------------: |
| **score** |   score   |                         分数                         |
| **shoot** | open_shoot| 是否玩家处于开放位置或无干扰 |
|          |    two    |               是否是两分尝试                |
|          |   three   |               是否是三分尝试               |
|          |  goal_in  |                     是否进球                      |
|          | hit_percent |                        命中率（大概是游戏实时计算的命中率）                       |
|          |   assist  |             玩家是否参与助攻             |
|          | inference |             玩家是否参与推断（待定）             |
|          |inference_degree|                   推断程度 （待定）                   |
| **steal** |   target   |             玩家是否是抢断的目标             |
|          | hit_percent |                        命中率（大概是游戏实时计算的命中率）                       |
|          |  success  |                     是否成功                      |
| **block** |   target   |             玩家是否是盖帽的目标             |
|          |expected_score| 主要由是否是两分或三分尝试和防守干扰的程度决定的被盖帽玩家的预期得分 |
|          | hit_percent |                        命中率（大概是游戏实时计算的命中率）                       |
|          |  success  |                     是否成功                      |
| **pickup** |  success  |                     是否成功                      |
|**rebound** |  success  |                     是否成功                      |
|**screen**  |           |                      是否在盯防                            |

#### 状态事件
状态事件如下表所示：

|     特征键     |                         描述                          |
| :------------: | :---------------------------------------------------: |
|**not_ball_clear**| 三分球没有出界，通常在转换拥有权时发生 |
|   **free_ball**   |               球不在任何玩家手中                |
|**attack_time_out**|                    进攻时间耗尽                    |
| **got_defended**  |                   玩家正在被防守                   |
| **out_of_defend** |                  玩家摆脱了防守                   |
|**no_defend_shoot**|              玩家在没有防守的情况下射击              |
|   **long_pass**   | 长传，这将导致玩家在游戏中变得僵硬 |
|   **pass_fail**   |                        传球失败                        |
"""

问题3:states状态包含大量冗余且无意义信息
有些特征可能是冗余的或对决策作用不大，例如“buff_key”和“buff_value”，而有些特征重复或非常相似，
例如，如果你知道哪个队伍拥有球，那么你可能不需要为每个玩家指定 “is_ball_owner”
以及可以计算每个玩家距离他的篮筐的距离，而不是他们的绝对位置
对于与篮球的相对位置等特征，应该使用极坐标系统代替笛卡尔坐标系统以获得代表性

----
关于ASN和HPN，以及我的构想，会在下一条回复中出现




我的构想如下，有任何不清楚的地方请随时提出疑问，有任何不合理的地方也请随时提出疑问：
#### 构建"Rewards网络"
首先从rewards出发，如果人为设计奖励函数，根据infos人为写一堆if-else是再蠢不过的事情，rewards很难自动调节。因此用神经网络实现rewards函数的想法就诞生了，思路如下:
- 目标：在环境返回的rewards的基础上，根据infos进行调整，使其奖励分配更加合理
- 输入：(env)rewards / (持续返回)状态事件 / (触发才返回，最小为0个，最大为1个)节点事件
- 输入：
* $R_t$-rewards部分:由env返回
* $O_t^{env}$-global_state部分:比如"attack_remain_time"/"match_remain_time"
* $a_t^i$-动作：上一次的动作
* $$


#### 在构想时，我已经对此提出一些疑惑：
问题1：如果rewards也使用神经网络，这个时候env就变成非稳态的，每次返回的rewards可能都不确定

#### 我对这些疑惑想了一些解决方法，不过很多可能很不实际，很不合理：

---

现在，请你对我的构想大胆质疑，构造一些使这个构想出现问题的情景，同时也可以针对其中可以优化和改进的地方提出具体的思路。